In [12]:
import os
# os.getenv("GROQ_API_KEY")
# GROQ_API_KEY is used to authenticate with the Groq API service.
# Make sure to set this environment variable in your .env file or system environment.
# Example: export GROQ_API_KEY="your_api_key_here"
# For security reasons, do not hardcode sensitive information like API keys directly in your code.
# Instead, use environment variables or secure vaults to manage them.

In [ ]:
# pip install arxiv
# !pip install tavily

In [14]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [15]:
api_wrapper = ArxivAPIWrapper(
    top_k_results=2,
    document_content_chars_max=500
)
arxiv = ArxivQueryRun(
    api_wrapper=api_wrapper,
    description="Query arXiv research papers"
)
print(arxiv.name)

arxiv


In [16]:
arxiv.invoke("What are the latest research papers on artificial intelligence?")

'Published: 2022-04-21\nTitle: Creative Problem Solving in Artificially Intelligent Agents: A Survey and Framework\nAuthors: Evana Gizzi, Lakshmi Nair, Sonia Chernova, Jivko Sinapov\nSummary: Creative Problem Solving (CPS) is a sub-area within Artificial Intelligence (AI) that focuses on methods for solving off-nominal, or anomalous problems in autonomous systems. Despite many advancements in planning and learning, resolving novel problems or adapting existing knowledge to a new context, especially in cases where the environment may change in unpredictable ways post deployment, remains a limiting factor in the safe and useful integration of intelligent systems. The emergence of increasingly autonomous systems dictates the necessity for AI agents to deal with environmental uncertainty through creativity. To stimulate further research in CPS, we present a definition and a framework of CPS, which we adopt to categorize existing AI methods in this field. Our framework consists of four main

In [17]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')


In [18]:
# Tavily search tool
from langchain_community.tools.tavily_search import TavilySearchResults
tavily = TavilySearchResults()

In [19]:
# combine all together
tools=[arxiv,tavily]

In [20]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="qwen/qwen3-32b")
# llm.invoke("whar is AI")

In [21]:
# llm.invoke("What is AI")

In [23]:
llm_with_tools=llm.bind_tools(tools=tools)

In [34]:
llm_with_tools.invoke("What is AI")

AIMessage(content='AI, or Artificial Intelligence, refers to systems or machines designed to perform tasks that typically require human intelligence. These tasks include learning from experience, reasoning, problem-solving, understanding natural language, recognizing patterns, and making decisions. \n\nAI can be categorized into:\n1. **Narrow AI**: Specialized for specific tasks (e.g., voice assistants like Siri, recommendation algorithms, or image recognition tools).\n2. **General AI**: A theoretical, human-like intelligence capable of handling any intellectual task (not yet achieved).\n\nAI powers technologies like self-driving cars, medical diagnostics, and personalized marketing. It relies on techniques such as machine learning, where systems improve performance through data exposure. While AI enhances efficiency, it also raises ethical questions about bias, privacy, and job displacement.', additional_kwargs={'reasoning_content': 'Okay, the user is asking "What is AI?" I need to pr

In [24]:
# STATE SCHEMEA
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage, HumanMessage #human message or ai message, added HumanMessage for initial input
from typing import Annotated #labeling
from langgraph.graph import add_messages

In [25]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages] #list of messages in the chat
    

In [26]:
from IPython.display import display, Image
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [ ]:
def tools_Calling(state:State):
    return {"message":[llm_with_tools.invoke(state["messages"])]}
# graph
builder=StateGraph(State)
builder.add_node("tool calling",tools_Calling)
builder.add_node("tools",ToolNode(tools))
#Edges
builder.add_edge(START,"tool calling")
builder.add_conditional_edges(
    "tool calling",
    #if the latest message (result) from assistant is a tool call -> tool_condtions routes to took
    #if the latest message (result) from assistant is a not a tool call -> tools_condtions routes to END
    tools_condition)

builder.add_edge("tools",END)
graph=builder.compile()
# display(Image(graph.get_graph().draw_mermaid_png()))


In [31]:
messages=graph.invoke({"messages": [HumanMessage(content="Hi,What is AI")]}) 
for msg in messages["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Hi,What is AI
================================== Ai Message ==================================

AI, or Artificial Intelligence, refers to systems or machines that demonstrate human-like intelligence. These systems can perform tasks such as learning from experience, understanding natural language, recognizing patterns, solving problems, and making decisions. AI encompasses various subfields, including machine learning, natural language processing, computer vision, and robotics. It is applied in diverse areas like healthcare, finance, transportation, and entertainment to automate processes, enhance efficiency, and provide insights through data analysis. If you'd like, I can provide more specific details or recent advancements in AI using available tools.


In [15]:
name=123345
s=name[::-1]
print(s)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\adity\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3667, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\adity\AppData\Local\Temp\ipykernel_10580\1745773863.py", line 2, in <module>
    s=name[::-1]
      ~~~~^^^^^^
TypeError: 'int' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\adity\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2176, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=tb_offset
    )
  File "c:\Users\adity\miniconda3\Lib\site-packages\IPython\core\ultratb.py", line 1182, in structured_traceback
    return FormattedTB.structured_traceback(
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        self, etype, evalue, etb, tb_offset, con